In [22]:
from os import rename
import pandas as pd

matches = pd.DataFrame({
    "match_id": [1,2,3,4],
    "date": ["2025-01-10","2025-01-12","2025-01-15","2025-01-20"],
    "home_team": ["Spain","Germany","Brazil","Italy"],
    "away_team": ["France","Spain","Argentina","Germany"],
    "home_score": [2,1,3,0],
    "away_score": [1,2,2,1]
})

rankings = pd.DataFrame({
    "team": ["Spain","Germany","Brazil","France","Italy","Argentina"],
    "fifa_rank": [2,3,1,5,8,4]
})

df = matches.merge(rankings, left_on="home_team", right_on="team", how="left")
df = df.rename(columns={"fifa_rank" : "home_rank"})
df = df.drop(columns="team")

print(df)

df = df.merge(rankings, left_on="away_team", right_on="team", how="left")
df = df.rename(columns={"fifa_rank" : "away_rank"})
df = df.drop(columns="team")

print(df)

df["result"] = df.apply(lambda x:
                        "HomeWin" if x.home_score > x.away_score else
                        "AwayWin" if x.home_score < x.away_score else
                        "Draw", axis=1
                        )
print(df)

pivot = pd.pivot_table(
    pd.concat([
        df[["home_team","home_score"]].rename(columns={"home_team" : "team", "home_score":"goals"}),
        df[["away_team","away_score"]].rename(columns={"away_team" : "team", "away_score":"goals"})
    ]),
    values="goals",
    index="team",
    aggfunc="sum",
    fill_value=0
)

print(pivot)

melted = pivot.reset_index().melt(id_vars="team",value_name="tgoals")
melted = melted.drop(columns="variable")
melted = melted.sort_values("tgoals", ascending=False)

print(melted)

df["goalDiff"] = (df["home_score"] - df['away_score']).abs()

print(df)

top2 = melted.sort_values("tgoals", ascending=False).head(2)

print(top2)

idk = df.groupby("result").size()
print(idk)



   match_id        date home_team  away_team  home_score  away_score  \
0         1  2025-01-10     Spain     France           2           1   
1         2  2025-01-12   Germany      Spain           1           2   
2         3  2025-01-15    Brazil  Argentina           3           2   
3         4  2025-01-20     Italy    Germany           0           1   

   home_rank  
0          2  
1          3  
2          1  
3          8  
   match_id        date home_team  away_team  home_score  away_score  \
0         1  2025-01-10     Spain     France           2           1   
1         2  2025-01-12   Germany      Spain           1           2   
2         3  2025-01-15    Brazil  Argentina           3           2   
3         4  2025-01-20     Italy    Germany           0           1   

   home_rank  away_rank  
0          2          5  
1          3          2  
2          1          4  
3          8          3  
   match_id        date home_team  away_team  home_score  away_score  \
0